In [5]:
!mkdir /tmp/learn-parquet

mkdir: /tmp/learn-parquet: File exists


In [42]:
from pyspark.sql import SparkSession
from datetime import datetime, date
from faker import Faker
import pandas as pd
from random import randint, choice

spark = SparkSession.builder.getOrCreate()

In [43]:
fake_salary = lambda : (randint(20000, 200000) // 1000 ) * 1000
fake_salary()

85000

In [44]:
fake_gender = lambda : choice(['man', 'woman', 'nonbinary', 'custom'])
fake_gender()

'custom'

In [53]:
fake = Faker()
data = [(fake.first_name(), fake.last_name(), fake.date(), fake_gender(), fake_salary()) for i in range(100)]
data[:5]

[('Christina', 'Tucker', '2020-05-29', 'woman', 188000),
 ('Brian', 'Lynn', '2014-12-18', 'man', 23000),
 ('Stephanie', 'Forbes', '1983-04-10', 'man', 191000),
 ('Richard', 'Harrison', '1970-06-16', 'woman', 50000),
 ('Kathy', 'Contreras', '2018-07-26', 'nonbinary', 155000)]

In [54]:
df = spark.createDataFrame(data[:5], schema=["firstname","lastname","dob","gender","salary"])
df

DataFrame[firstname: string, lastname: string, dob: string, gender: string, salary: bigint]

This will create one partition per item

In [56]:
df.write.parquet("/tmp/learn-parquet/people.parquet")

In [57]:
!ls /tmp/learn-parquet/people.parquet

_SUCCESS
part-00000-2eaa9efb-a128-4049-9587-f5b35f466b88-c000.snappy.parquet
part-00001-2eaa9efb-a128-4049-9587-f5b35f466b88-c000.snappy.parquet
part-00003-2eaa9efb-a128-4049-9587-f5b35f466b88-c000.snappy.parquet
part-00005-2eaa9efb-a128-4049-9587-f5b35f466b88-c000.snappy.parquet
part-00007-2eaa9efb-a128-4049-9587-f5b35f466b88-c000.snappy.parquet
part-00009-2eaa9efb-a128-4049-9587-f5b35f466b88-c000.snappy.parquet


In [58]:
df = spark.createDataFrame(data, ["firstname","lastname","dob","gender","salary"])
df.write.parquet("/tmp/learn-parquet/people.parquet2")

This creates ten partitions for some reason.. (there are 100 items)

In [59]:
!ls /tmp/learn-parquet/people.parquet2

_SUCCESS
part-00000-110f07d5-a9b8-4bca-8541-a3484039a76f-c000.snappy.parquet
part-00001-110f07d5-a9b8-4bca-8541-a3484039a76f-c000.snappy.parquet
part-00002-110f07d5-a9b8-4bca-8541-a3484039a76f-c000.snappy.parquet
part-00003-110f07d5-a9b8-4bca-8541-a3484039a76f-c000.snappy.parquet
part-00004-110f07d5-a9b8-4bca-8541-a3484039a76f-c000.snappy.parquet
part-00005-110f07d5-a9b8-4bca-8541-a3484039a76f-c000.snappy.parquet
part-00006-110f07d5-a9b8-4bca-8541-a3484039a76f-c000.snappy.parquet
part-00007-110f07d5-a9b8-4bca-8541-a3484039a76f-c000.snappy.parquet
part-00008-110f07d5-a9b8-4bca-8541-a3484039a76f-c000.snappy.parquet
part-00009-110f07d5-a9b8-4bca-8541-a3484039a76f-c000.snappy.parquet


In [72]:
rdd = spark.sparkContext.parallelize(data, 5)
df = spark.createDataFrame(rdd, ["firstname","lastname","dob","gender","salary"])
df.write.parquet("/tmp/learn-parquet/people.parquet3")

This creates 5 partitions because we partitioned the data into 5 partitions.

In [73]:
!ls /tmp/learn-parquet/people.parquet3

_SUCCESS
part-00000-493be267-8a8c-4d0f-ae90-f8f70a827ba6-c000.snappy.parquet
part-00001-493be267-8a8c-4d0f-ae90-f8f70a827ba6-c000.snappy.parquet
part-00002-493be267-8a8c-4d0f-ae90-f8f70a827ba6-c000.snappy.parquet
part-00003-493be267-8a8c-4d0f-ae90-f8f70a827ba6-c000.snappy.parquet
part-00004-493be267-8a8c-4d0f-ae90-f8f70a827ba6-c000.snappy.parquet
